In [19]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("sample_cocoeval.ipynb"))))

from data.yolo_dataset import YoloDataset

import torch
import tqdm

In [14]:
dataset = YoloDataset

In [20]:
def train(
        model,
        train_loader,
        loss_func,
        optimizer,
        optim_option,
        model_option,
        device,
        epoch,
        logger
        ):
    model.train()

    scales = torch.tensor(model_option["YOLOv3"]["SCALES"]).to(device)       ## [13, 26, 52]
    anchors = torch.tensor(model_option["YOLOv3"]["ANCHORS"]).to(device)

    for i, batch_input in enumerate(tqdm(train_loader, desc="train")):
        n_iteration = (optim_option["OPTIMIZER"]["ITERS_PER_EPOCH"] * epoch) + i

        batch_img = batch_input["img"].to(device)
        batch_label = [label.to(device) for label in batch_input["label_map"]]
        
        #################
        ##  FORWARDING ##
        #################
        pred = model(batch_img)                                                       ### batch_img: tensor(   N, 3, 608, 608) . . . . . . . . . . . N = batch_size
        loss = ( loss_func(pred[0], batch_label[0], scales[0], anchors=anchors[0])    ######## pred: tensor(3, N, 3, S, S, 1 + 4 + class_offset) . . S = scale_size
               + loss_func(pred[1], batch_label[1], scales[1], anchors=anchors[1])    # batch_label: tensor(3, N, 3, S, S, 1 + 4 + class_offset)
               + loss_func(pred[2], batch_label[2], scales[2], anchors=anchors[2]) )  ##### anchors: tensor(3,    3,       2) . . . is list of pairs(anch_w, anch_h)
        loss /= 3

        logger.add_scalar('train/loss', loss.item(), n_iteration)

        # print(f"loss: {loss}")

        #################
        ## BACKWARDING ##
        #################
        loss.backward()
        optimizer.step()

        torch.cuda.empty_cache()


In [21]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

## https://www.programcreek.com/python/example/88588/pycocotools.cocoeval.COCOeval

In [ ]:
import torchvision

class COCOevaluator(object):
    def __init__(self, dataloader, iou_type="bbox"):
        super(COCOevaluator, self).__init__()

        self.cocogt = self.create_cocogt(dataloader.dataset)
        self.cocodt = []
        self.evaluator = COCOeval(cocoGt=self.cocogt, iouType=iou_type)


    def update_preds(self, pred):

        self.cocodt.append(pred)


    def create_cocogt(self, dataset):
        for _ in range(10):
            if isinstance(dataset, torchvision.datasets.CocoDetection):
                break
            if isinstance(dataset, torch.utils.data.Subset):
                dataset = dataset.dataset

        if isinstance(dataset, torchvision.datasets.CocoDetection):
            return dataset.coco

        cocogt = COCO()

        ann_id = 1      ## There was an issue: WHY `ann_id` should be initialized to 1 ----- https://github.com/pytorch/vision/issues/1530
                        ## In short, the list of class IDs is used as BOOLEAN values
                        ##        => So if `ann_id` is initialized to 0, then the `0th ID` is interpreted as a NEGATIVE

        dataset = {'images': [], 'categories': [], 'annotations': []}
        categories = set()
        for img_idx in range(len(dataset)):
            # find better way to get target
            # targets = dataset.get_annotations(img_idx)
            img, labels, img_path, _ = dataset[img_idx]           ## __getitem__ RETURN: (img, labels, img_path, label_maps)

            img_dict = {}

            image_id = os.path.splitext(img_path)[0]    ## e.g. "daecheon_20201113_0000_011"
            img_dict['id'] = image_id
            img_dict['height'] = img.shape[-2]
            img_dict['width'] = img.shape[-1]

            dataset['images'].append(img_dict)

            labels = labels[..., 0].tolist()
            bboxes = labels[..., 1:5]
            bboxes[..., :2] -= (bboxes[..., 2:] / 2)
            bboxes = bboxes.tolist()

            areas = (bboxes[..., 2:3] * bboxes[..., 3:4]).tolist()    #######################
            iscrowd = targets['iscrowd'].tolist()                     #######################

            num_objs = len(bboxes)
            for i in range(num_objs):
                ann = {}
                ann['image_id'] = image_id
                ann['bbox'] = bboxes[i]
                ann['category_id'] = labels[i]

                categories.add(labels[i])

                ann['area'] = areas[i]
                ann['iscrowd'] = iscrowd[i]
                ann['id'] = ann_id

                dataset['annotations'].append(ann)
                ann_id += 1

        dataset['categories'] = [{'id': i} for i in sorted(categories)]
        cocogt.dataset = dataset
        cocogt.createIndex()
        
        return cocogt


    def evaluation(self):
        self.evaluator.evaluate()
        self.evaluator.accumulate()
        self.evaluator.summarize()

        mean_average_precision = self.evaluator.stats[0].item()

        return mean_average_precision

In [ ]:
def valid(
        model,
        valid_loader,
        model_option,
        device,
        epoch,
        logger
        ):
    model.eval()

    coco_evaluator = COCOevaluator(valid_loader.dataset)

    for i, batch_input in enumerate(tqdm(valid_loader, desc="valid")):
        batch_img = batch_input["img"].to(device)
        batch_label = [label.to(device) for label in batch_input["label_map"]]
        batch_size = len(batch_input["img_path"])
        
        pred = model(batch_img)
        # surpressed_pred = NMS(pred, batch_size)
        coco_evaluator.update_preds(pred)


    ## Examine Accuracy: mean Average Precision
    mean_average_precision = coco_evaluator.evaluation()
    logger.add_scalar('test/map', mean_average_precision, epoch)

    return mean_average_precision
